In [1]:
cd ..

/home/agrigorev/notebooks/home-depot/homedepot


In [2]:
import Google_spell_check

In [3]:
import numpy as np
import pandas as pd

In [4]:
from time import time

In [5]:
import re

from unidecode import unidecode
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

stemmer = PorterStemmer()

In [6]:
root_path = '/home/agrigorev/notebooks/home-depot/input'

df_train = pd.read_csv(root_path + '/train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv(root_path + '/test.csv', encoding="ISO-8859-1")
df_pro_desc = pd.read_csv(root_path + '/product_descriptions.csv')

df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

In [7]:
df_products = df_all.groupby('product_uid')[['product_title', 'product_uid']].head(n=1)
df_products.fillna('', inplace=1)

df_all.drop(['product_title'], axis=1, inplace=1)

df_products = df_products.merge(df_pro_desc, on='product_uid')

In [8]:
df_attr = pd.read_csv(root_path + '/attributes.csv', encoding='utf-8')
df_attr.name = df_attr.name.str.lower()

In [9]:
spell_check = Google_spell_check.spell_check_dict

def get_or_identity(query):
    return spell_check.get(query, query)

df_all.search_term = df_all.search_term.apply(get_or_identity)

In [10]:
def parse_correction(line):
    return line.strip().split('->')

with open('one-word-corrections.txt', 'r') as f:
    corrections = [parse_correction(s) for s in f.readlines()]
    corrections = [s for s in corrections if len(s) == 2]
    corrections = {k: v.split(' ') for (k, v) in corrections}

with open('many-words-corrections.txt', 'r') as f:
    multiword_corrections = [parse_correction(s) for s in f.readlines()]
    multiword_corrections = [s for s in multiword_corrections if len(s) == 2]

In [11]:
stopwords = {'a', 'from', 'only', 'do', 'with', 'the', 'and', 'for', 'up', 'to', 'be', 'per',
            'th', 'on', 'what', 'that'}

In [12]:
def str_stem(s): 
    if not isinstance(s, (str, unicode)):
        return []

    if isinstance(s, str):
        s = unicode(s.decode('utf-8'))

    # some title edits?
    s = s.replace("&quot;"," ")
    s = s.replace(u"è_"," ")
    s = s.replace(u"å¡"," ")
    s = s.replace(u"Û"," ")
    s = s.replace(u"åÊ"," ")
    s = s.replace(u"ÛÒ"," ")
    s = s.replace(u"Ûª"," ")
    s = s.replace(u"ÛÜ"," ")
    s = s.replace(u"Û÷"," ")
    s = s.replace(u"ÈÀ"," ")
    s = s.replace(u"ã¢"," ")        
    s = s.replace(u"Ã¥Â¡"," ")
    s = s.replace(u"ã¨_"," ")

    s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A?
    s = s.replace("U.S."," US ")
    s = s.lower()

    s = re.sub(r"(\w)\.(\w)", r"\1 \2", s)
    s = s.replace("&#39;s"," ")

    s = s.replace("  "," ")
    s = s.replace(",","") #could be number / segment later?
    s = s.replace("$"," ")
    s = s.replace("+"," plus ")
    s = s.replace(";"," ")
    s = s.replace(":"," ")
    s = s.replace("&amp;"," ")
    s = s.replace("&amp"," ")
    s = s.replace("?"," ")
    s = s.replace("-"," ")
    s = s.replace("#"," ")
    s = s.replace("("," ")
    s = s.replace(")"," ")
    s = s.replace("//","/")
    s = s.replace("..",".")
    s = s.replace(" / "," ovr ")
    s = s.replace(" \\ "," ")
    s = s.replace("."," . ")

    s = re.sub(r"(^\.|/)", r" ", s)
    s = re.sub(r"(\.|/)$", r" ", s)
    s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
    s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
    s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
    s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)

    s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
    s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in ", s)
    s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft ", s)
    s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb ", s)
    s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sqft ", s)
    s = re.sub(r"([0-9]+)( *)(square|sq)\.?", r"\1sq ", s)
    s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cuft ", s)
    s = re.sub(r"([0-9]+)( *)(cubic|cu)\.?", r"\1cu ", s)
    s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal ", s)
    s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz ", s)
    s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm ", s)
    s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm ", s)
    s = re.sub(r"([0-9]+)( *)(deg|degrees|degree)\.?", r"\1deg ", s)
    s = re.sub(r"([0-9]+)( *)(volts|volt|v)\.?", r"\1volt ", s)
    s = re.sub(r"([0-9]+)( *)(watts|watt|w)\.?", r"\1watt ", s)
    s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp ", s)

    s = re.sub(r'depot.com/search=', '', s)
    s = re.sub(r'pilers,needlenose', 'pliers, needle nose', s)    
    
    s=s.replace("ttt","tt")    
    s=s.replace("lll","ll") 
    s=s.replace("nnn","nn") 
    s=s.replace("rrr","rr") 
    s=s.replace("sss","ss") 
    s=s.replace("zzz","zz")
    s=s.replace("ccc","cc")
    s=s.replace("eee","ee")
    
    s=s.replace("acccessories","accessories")
    s=re.sub(r'\bscott\b', 'scotts', s) #brand
    s=re.sub(r'\borgainzer\b', 'organizer', s)
    s=re.sub(r'\bshark bite\b', 'sharkbite',s)
    
    s=s.replace("hinges with pishinges with pins","hinges with pins")    
    s=s.replace("virtue usa","virtu usa")
    s=re.sub('outdoor(?=[a-rt-z])', 'outdoor ', s)
    s=re.sub(r'\bdim able\b',"dimmable", s) 
    s=re.sub(r'\blink able\b',"linkable", s)
    s=re.sub(r'\bm aple\b',"maple", s)
    s=s.replace("aire acondicionado", "air conditioner")
    s=s.replace("borsh in dishwasher", "bosch dishwasher")
    s=re.sub(r'\bapt size\b','appartment size', s)
    s=re.sub(r'\barm[e|o]r max\b','armormax', s)
    s=re.sub(r' ss ',' stainless steel ', s)
    s=re.sub(r'\bmay tag\b','maytag', s)
    s=re.sub(r'\bback blash\b','backsplash', s)
    s=re.sub(r'\bbum boo\b','bamboo', s)
    s=re.sub(r'(?<=[0-9] )but\b','btu', s)
    s=re.sub(r'\bcharbroi l\b','charbroil', s)
    s=re.sub(r'\bair cond[it]*\b','air conditioner', s)
    s=re.sub(r'\bscrew conn\b','screw connector', s)
    s=re.sub(r'\bblack decker\b','black and decker', s)
    s=re.sub(r'\bchristmas din\b','christmas dinosaur', s)
    s=re.sub(r'\bdoug fir\b','douglas fir', s)
    s=re.sub(r'\belephant ear\b','elephant ears', s)
    s=re.sub(r'\bt emp gauge\b','temperature gauge', s)
    s=re.sub(r'\bsika felx\b','sikaflex', s)
    s=re.sub(r'\bsquare d\b', 'squared', s)
    s=re.sub(r'\bbehring\b', 'behr', s)
    s=re.sub(r'\bcam\b', 'camera', s)
    s=re.sub(r'\bjuke box\b', 'jukebox', s)
    s=re.sub(r'\brust o leum\b', 'rust oleum', s)
    s=re.sub(r'\bx mas\b', 'christmas', s)
    s=re.sub(r'\bmeld wen\b', 'jeld wen', s)
    s=re.sub(r'\bg e\b', 'ge', s)
    s=re.sub(r'\bmirr edge\b', 'mirredge', s)
    s=re.sub(r'\bx ontrol\b', 'control', s)
    s=re.sub(r'\boutler s\b', 'outlets', s)
    s=re.sub(r'\bpeep hole', 'peephole', s)
    s=re.sub(r'\bwater pik\b', 'waterpik', s)
    s=re.sub(r'\bwaterpi k\b', 'waterpik', s)
    s=re.sub(r'\bplex[iy] glass\b', 'plexiglass', s)
    s=re.sub(r'\bsheet rock\b', 'sheetrock',s)
    s=re.sub(r'\bgen purp\b', 'general purpose',s)
    s=re.sub(r'\bquicker crete\b', 'quikrete',s)
    s=re.sub(r'\bref ridge\b', 'refrigerator',s)
    s=re.sub(r'\bshark bite\b', 'sharkbite',s)
    s=re.sub(r'\buni door\b', 'unidoor',s)
    s=re.sub(r'\bair tit\b','airtight', s)
    s=re.sub(r'\bde walt\b','dewalt', s)
    s=re.sub(r'\bwaterpi k\b','waterpik', s)
    s=re.sub(r'\bsaw za(ll|w)\b','sawzall', s)
    s=re.sub(r'\blg elec\b', 'lg', s)
    s=re.sub(r'\bhumming bird\b', 'hummingbird', s)
    s=re.sub(r'\bde ice(?=r|\b)', 'deice',s)  
    s=re.sub(r'\bliquid nail\b', 'liquid nails', s)  
    s=re.sub(r'\bdeck over\b','deckover', s)
    s=re.sub(r'\bcounter sink(?=s|\b)','countersink', s)
    s=re.sub(r'\bpipes line(?=s|\b)','pipeline', s)
    s=re.sub(r'\bbook case(?=s|\b)','bookcase', s)
    s=re.sub(r'\bwalkie talkie\b','2 pair radio', s)
    s=re.sub(r'(?<=^)ks\b', 'kwikset',s)

    
    s=re.sub(r'(?<=[1-9]) pac\b', 'pack', s)
 
    s=re.sub(r'\bcfl bulbs\b', 'cfl light bulbs', s)
    s=re.sub(r' cfl(?=$)', ' cfl light bulb', s)
    s=re.sub(r'candelabra cfl 4 pack', 'candelabra cfl light bulb 4 pack', s)
    s=re.sub(r'\bthhn(?=$|\ [0-9]|\ [a-rtuvx-z])', 'thhn wire', s)
    s=re.sub(r'\bplay ground\b', 'playground',s)
    s=re.sub(r'\bemt\b', 'emt electrical metallic tube',s)
    s=re.sub(r'\boutdoor dining se\b', 'outdoor dining set',s)    

    s = s.replace(" x ", " xby ")
    s = s.replace("*", " xby ")
    s = s.replace(" by ", " xby")
    s = s.replace("x0", " xby 0")
    s = s.replace("x1", " xby 1")
    s = s.replace("x2", " xby 2")
    s = s.replace("x3", " xby 3")
    s = s.replace("x4", " xby 4")
    s = s.replace("x5", " xby 5")
    s = s.replace("x6", " xby 6")
    s = s.replace("x7", " xby 7")
    s = s.replace("x8", " xby 8")
    s = s.replace("x9", " xby 9")
    s = s.replace("0x", "0 xby ")
    s = s.replace("1x", "1 xby ")
    s = s.replace("2x", "2 xby ")
    s = s.replace("3x", "3 xby ")
    s = s.replace("4x", "4 xby ")
    s = s.replace("5x", "5 xby ")
    s = s.replace("6x", "6 xby ")
    s = s.replace("7x", "7 xby ")
    s = s.replace("8x", "8 xby ")
    s = s.replace("9x", "9 xby ")

    s = s.replace("&"," ")
    s = s.replace("'"," ")
    s = s.replace("  "," ")
    s = s.replace(" . "," ")

    s = unidecode(s.lower())
    
    for orig, repl in multiword_corrections:
        s = s.replace(orig, repl)
    
    result = []

    
    for z in s.split(" "):
        z = z.strip()
        if not z:
            continue
        if z in stopwords:
            continue

        if z in corrections:
            result.extend(corrections[z])
        else:
            result.append(z)

    return [stemmer.stem(z) for z in result]

In [13]:
from multiprocessing import Pool
pool = Pool(processes=8) 

In [14]:
def process_parallel(pool, series, function):
    return pool.map(function, series)

In [15]:
t0 = time()

df_all['search_term'] = process_parallel(pool, df_all['search_term'], str_stem)
df_products['product_title'] = process_parallel(pool, df_products['product_title'], str_stem)
df_products['product_description'] = process_parallel(pool, df_products['product_description'], str_stem)
df_attr['value'] = process_parallel(pool, df_attr['value'], str_stem)

print 'took %0.5fs.' % (time() - t0)

took 330.59844s.


In [17]:
all_text = []

df_all['search_term'].apply(all_text.append)
df_products['product_title'].apply(all_text.append)
df_products['product_description'].apply(all_text.append)
df_attr['value'].apply(all_text.append)

len(all_text)

2534419

In [18]:
from gensim.models import Word2Vec

num_features = 300    # Word vector dimensionality
min_word_count = 10   # Minimum word count
num_workers = 8     # Number of threads to run in parallel
context = 3          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

Couldn't import dot_parser, loading of dot files will not be possible.


In [19]:
t0 = time()

model = Word2Vec(all_text, workers=num_workers, size=num_features, min_count=min_word_count,
                 window=context, sample=downsampling, seed=1)

print 'took %0.5fs.' % (time() - t0)

took 109.76043s.


In [20]:
model.save('home_depot_w2v_clean.bin')